In [ ]:
import pandas as pd

In [ ]:
df=pd.read_pickle("/content/2_CarPricesData.pkl")
df

,Age,KM,Weight,HP,MetColor,CC,Doors,Price
0,23.0,46986,1165.0,90,1,2000.0,3,13500
1,23.0,72937,1165.0,90,1,2000.0,3,13750
2,24.0,41711,1165.0,90,1,2000.0,3,13950
3,26.0,48000,1165.0,90,0,2000.0,3,14950
4,30.0,38500,1170.0,90,0,2000.0,3,13750
...,...,...,...,...,...,...,...,...
1431,69.0,20544,1025.0,86,1,1300.0,3,7500
1432,72.0,19000,1015.0,86,0,1300.0,3,10845
1433,71.0,17016,1015.0,86,0,1300.0,3,8500
1434,70.0,16916,1015.0,86,1,1300.0,3,7250


In [ ]:
df.isnull().sum()

,0
Age,0
KM,0
Weight,0
HP,0
MetColor,0
CC,0
Doors,0
Price,0


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.dtypes

,0
Age,float64
KM,int64
Weight,float64
HP,int64
MetColor,int64
CC,float64
Doors,int64
Price,int64


In [ ]:
df.columns

Index(['Age', 'KM', 'Weight', 'HP', 'MetColor', 'CC', 'Doors', 'Price'], dtype='object')

In [ ]:
df.shape

(1435, 8)

In [ ]:
numeric_cols = ['Age', 'KM', 'Weight', 'HP', 'MetColor', 'CC', 'Doors', 'Price']

outlier_counts = {}

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_counts[col] = len(outliers)

for col, count in outlier_counts.items():
    print(f"{col}: {count} outliers")

Age: 7 outliers
KM: 49 outliers
Weight: 65 outliers
HP: 11 outliers
MetColor: 0 outliers
CC: 121 outliers
Doors: 0 outliers
Price: 109 outliers


In [ ]:
def remove_outliers_zscore(df, column, threshold=3):
    mean = df[column].mean()
    std = df[column].std()
    z_scores = (df[column] - mean) / std
    df_cleaned = df[(z_scores < threshold) & (z_scores > -threshold)].copy()
    return df_cleaned

for col in numeric_cols:
    df = remove_outliers_zscore(df, col)

df.shape

(1354, 8)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
df[['Age','KM','Weight','HP','CC']] = sc.fit_transform(df[['Age','KM','Weight','HP','CC']])
df.head()

,Age,KM,Weight,HP,MetColor,CC,Doors,Price
0,0.278481,0.260107,0.804878,0.446809,1,1.0,3,13500
1,0.278481,0.403771,0.804878,0.446809,1,1.0,3,13750
2,0.291139,0.230905,0.804878,0.446809,1,1.0,3,13950
3,0.316456,0.265721,0.804878,0.446809,0,1.0,3,14950
4,0.367089,0.213129,0.829268,0.446809,0,1.0,3,13750


In [ ]:
X=df.drop('Price',axis=1)
y=df['Price']

In [ ]:
X.shape

(1354, 7)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train.shape

(1083, 7)

In [ ]:
X_test.shape

(271, 7)

In [ ]:
X_train.iloc[0]

,897
Age,0.822785
KM,0.332152
Weight,0.073171
HP,0.361702
MetColor,1.000000
CC,0.000000
Doors,3.000000


In [ ]:
from keras import Sequential
from keras.layers import Dense, Dropout
model=Sequential()
model.add(Dense(units=16,input_dim=7,kernel_initializer='random_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=32,kernel_initializer='he_uniform',activation='relu',name='H1'))
model.add(Dropout(0.4))
model.add(Dense(units=64,kernel_initializer='he_uniform',activation='relu',name='H2'))
model.add(Dropout(0.2))
model.add(Dense(units=32,kernel_initializer='he_uniform',activation='relu',name='H3'))
model.add(Dropout(0.5))
model.add(Dense(units=32,kernel_initializer='he_uniform',activation='relu',name='H4'))
model.add(Dense(units=1,kernel_initializer='he_uniform'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
'''model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])
model.fit(X_train,y_train,epochs=20,batch_size=40,verbose=1,validation_split=0.3)'''

"model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])\nmodel.fit(X_train,y_train,epochs=20,batch_size=40,verbose=1,validation_split=0.3)"

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense

early_stop=EarlyStopping(monitor='val_loss',verbose=1,patience=7,restore_best_weights=True)

checkpoint=ModelCheckpoint(filepath='best_model.keras',monitor='val_loss',verbose=1,save_best_only=True)

In [ ]:
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])

In [ ]:
history = model.fit(X_train, y_train, validation_split = 0.2, batch_size=20,epochs = 100, callbacks = [early_stop, checkpoint], verbose = 1)

Epoch 1/100
37/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 117447408.0000 - mse: 117447408.0000
Epoch 1: val_loss improved from inf to 111158864.00000, saving model to best_model.keras
44/44 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - loss: 117137832.0000 - mse: 117137832.0000 - val_loss: 111158864.0000 - val_mse: 111158864.0000
Epoch 2/100
33/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 115940032.0000 - mse: 115940032.0000
Epoch 2: val_loss improved from 111158864.00000 to 109123352.00000, saving model to best_model.keras
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 115653056.0000 - mse: 115653056.0000 - val_loss: 109123352.0000 - val_mse: 109123352.0000
Epoch 3/100
40/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 108390688.0000 - mse: 108390688.0000
Epoch 3: val_loss improved from 109123352.00000 to 75004616.00000, saving model to best_model.keras
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 107761712.0000 - mse: 107761712.0000 - val_loss: 75004616.0000 - val_mse: 75004608.0000
Epoch 4/100

In [ ]:
loss, accuracy = model.evaluate(X_test, X_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 96530144.0000 - mse: 96530144.0000 
Test Loss: 96482648.0000
Test Accuracy: 96482648.0000
